Cost Estimations pg141

In [44]:
import numpy as np
from scipy.stats import norm

In [45]:
def normal_loss_standard(x):
    return norm.pdf(x)-x*(1-norm.cdf(x))

In [46]:
k = 1000
h = 1.25
b =50
time = 100
d_mu = 100
d_std = 25
R = 4
alpha = 0.95

In [47]:
L, L_std = 4, 1
z = norm.ppf(alpha)
x_std = np.sqrt((R+L)*d_std**2+L_std**2*d_mu**2)
Ss = np.round(x_std*z).astype(int)
Cs = 1/2 * d_mu * R
Is = d_mu * L
S = Ss + 2 * Cs + Is
d = np.maximum(np.random.normal(d_mu, d_std, time).round(0).astype(int), 0)

In [48]:
p = np.zeros(time)
p[0]=S-d[0]/2

In [49]:
transit = np.zeros((time, L+ 5*L_std + 1), dtype=int)
transit[0,L] = d[0]

In [50]:
hand = np.zeros(time, dtype=int)
transit = np.zeros((time, L+R), dtype=int)

In [58]:
stockout_period = np.full(time, False, dtype=bool)
stockout_cycle = []
unit_shorts = np.zeros(time)

In [59]:
c_k = k
c_h = h*p[0]
c_b = 0

In [65]:
for t in range(1,time):
    if transit[t-1,0]>0:
        stockout_cycle.append(stockout_period[t-1])
    unit_shorts[t] = max(0,d[t] - max(0, hand[t-1] + transit[t-1,0]))
    hand[t] = hand[t-1] - d[t] + transit[t-1,0]
    stockout_period[t] = hand[t] < 0
    transit[t, :-1] = transit[t-1, 1:]
    if t%R==0:
        actual_L = int(round(max(np.random.normal(L,L_std), 0 ), 0))
        net = hand[t] + transit[t].sum()
        transit[t,actual_L] = S -net
        c_k +=k
    if hand[t] > 0:
        p[t] = (hand[t-1] + transit[t-1,0] + hand[t]/2)
    else:
        p[t] = max(hand[t-1] + transit[t-1,0], 0)**2/max(d[t],1)/2
    c_h += h*p[t]
    c_b += b*unit_shorts[t]

In [66]:
def cost(h,d_mu,R,z,x_std,k,b):
    return h*(d_mu*R/2 + x_std*z) + k/R + b*x_std*normal_loss_standard(z)/R
print('Model: \t\t', round(cost(h,d_mu,R,z,x_std,k,b), 1))
print('Simulation: \t', round((c_h+c_b+c_k)/time,1))

Model: 		 783.8
Simulation: 	 1694.9
